In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf

In [2]:
# Define the paths to the data directories
training_dir = r'D:\sd\FSOCO\training'
validation_dir = r'D:\sd\FSOCO\validation'
testing_dir = r'D:\sd\FSOCO\testing'

# Define the image size
IMAGE_SIZE = 224

# Define the batch size
BATCH_SIZE = 50

# Define the number of epochs
NUM_EPOCHS = 50

# Define the learning rate
LEARNING_RATE = 0.001

# Define a dictionary that maps class names to integer values
class_labels = {
    'blue_cone': 0,
    'yellow_cone': 1,
    'orange_cone': 2,
    'large_orange_cone': 3,
    'unknown_cone' : 4
}

# Define the number of training samples
num_training_samples = sum([len(files) for r, d, files in os.walk(training_dir)])

# Define the number of validation samples
num_validation_samples = sum([len(files) for r, d, files in os.walk(validation_dir)])

# Define the number of testing samples
num_testing_samples = sum([len(files) for r, d, files in os.walk(testing_dir)])

In [9]:
# Load the training data and annotations
training_images = []
training_annotations = []

for subdir, dirs, files in os.walk(training_dir):
    for file in files:
        if file.endswith('.png') or file.endswith('jpg'): #change later to jpg when using main dataset
            # Get the image path
            img_path = os.path.join(subdir, file)

            # Get the annotation path
            annotation_path = r'D:\sd\FSOCO\training\training_anns.csv' #can remove the r by switching the back slash to a forward slash.

            # Read the image and annotation file
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
            training_images.append(img)

            with open(annotation_path, 'r') as f:
                next(f)  # Skip the first line (header)
                annotation_lines = f.readlines()

            annotation = []

            for line in annotation_lines:
                line = line.strip().split(',')
                filename = line[0]
                class_name = line[1]
                x_min = int(line[2])
                y_min = int(line[3])
                x_max = int(line[4])
                y_max = int(line[5])
                class_id = class_labels[class_name]
                annotation.append([x_min, y_min, x_max, y_max, class_id])

            training_annotations.append(annotation)

# Convert the data to numpy arrays
training_images = np.array(training_images)
training_annotations = np.array(training_annotations)

In [4]:
# Load the validation data and annotations
validation_images = []
validation_annotations = []

for subdir, dirs, files in os.walk(validation_dir):
    for file in files:
        if file.endswith('.png') or file.endswith('.jpg'):
            # Get the image path
            img_path = os.path.join(subdir, file)
            
            # Get the annotation path
            annotation_path = r'D:\sd\FSOCO\validation\validation.csv'
            
            # Read the image and annotation file
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
            validation_images.append(img)
            
            with open(annotation_path, 'r') as f:
                next(f)  # Skip the first line (header)
                annotation_lines = f.readlines()
                
            annotation = []
            
            for line in annotation_lines:
                line = line.strip().split(',')
                filename = line[0]
                class_name = line[1]
                x_min = int(line[2])
                y_min = int(line[3])
                x_max = int(line[4])
                y_max = int(line[5])
                class_id = class_labels[class_name]
                annotation.append([x_min, y_min, x_max, y_max, class_id])
                
            validation_annotations.append(annotation)

# Convert the data to numpy arrays
validation_images = np.array(validation_images)
validation_annotations = np.array(validation_annotations)

In [10]:
# Load the testing data and annotations
testing_images = []
testing_annotations = []

for subdir, dirs, files in os.walk(testing_dir):
    for file in files:
        if file.endswith('.jpg') or file.endswith('.png'): #same as train set
            # Get the image path
            img_path = os.path.join(subdir, file)
            
            # Get the annotation path
            annotation_path = r'D:\sd\FSOCO\testing\testing_anns.csv'
            
            # Read the image and annotation file
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
            testing_images.append(img)
            
            with open(annotation_path, 'r') as f:
                next(f)  # Skip the first line (header)
                annotation_lines = f.readlines()
                
            annotation = []
            
            for line in annotation_lines:
                line = line.strip().split(',')
                filename = line[0]
                class_name = line[1]
                x_min = int(line[2])
                y_min = int(line[3])
                x_max = int(line[4])
                y_max = int(line[5])
                class_id = class_labels[class_name]
                annotation.append([x_min, y_min, x_max, y_max, class_id])
                
            testing_annotations.append(annotation)

# Convert the data to numpy arrays
testing_images = np.array(testing_images)
testing_annotations = np.array(testing_annotations)


In [11]:
#Normalize the image data
training_images = training_images / 255.0
validation_images = validation_images / 255.0
testing_images = testing_images / 255.0

In [12]:
#Print the shape of the training, validation, and testing data#
print('Training data shape:', training_images.shape)
print('Training annotations shape:', training_annotations.shape)
print('Validation data shape:', validation_images.shape)
print('Validation annotations shape:', validation_annotations.shape)
print('Testing data shape:', testing_images.shape)
print('Testing annotations shape:', testing_annotations.shape)

Training data shape: (233, 224, 224, 3)
Training annotations shape: (233, 5128, 5)
Validation data shape: (186, 224, 224, 3)
Validation annotations shape: (186, 5011, 5)
Testing data shape: (169, 224, 224, 3)
Testing annotations shape: (169, 4879, 5)


In [13]:
# define the training, validation, and testing datasets
train_ds = tf.data.Dataset.from_tensor_slices((training_images, training_annotations))
train_ds = train_ds.shuffle(buffer_size=len(training_images)).batch(BATCH_SIZE)

val_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_annotations))
val_ds = val_ds.batch(BATCH_SIZE)

test_ds = tf.data.Dataset.from_tensor_slices((testing_images, testing_annotations))
test_ds = test_ds.batch(BATCH_SIZE)

In [ ]:
import tensorflow as tf

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(5128*5, activation='softmax'),
    tf.keras.layers.Reshape((5128, 5))
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(training_images, training_annotations, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE,
                    validation_data=(validation_images, validation_annotations))

# Evaluate the model
test_loss, test_acc = model.evaluate(testing_images, testing_annotations)

# Make predictions on the test set
new_predictions = model.predict(testing_images)

# Print out the accuracy of the model
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(testing_annotations, new_predictions)
print(f"Accuracy of the model on the test set: {accuracy}")

# Save the model
model.save('models')